In [1]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [5]:
df=pd.read_csv('../data/processed_data.csv',usecols=lambda column:not column.startswith('Unnamed'))
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,614,Spain,Female,40,3,113348.50,1,1,1,77789.01,0
1,758,France,Female,34,1,154139.45,1,1,1,60728.89,0
2,541,Germany,Female,39,9,100116.67,1,1,1,199808.10,1
3,481,France,NotSpecified,37,8,152303.66,2,1,1,175082.20,0
4,711,France,Female,37,8,113899.92,1,0,0,80215.20,0


## Models Training
### Let's start by splitting our dataframe into separate objects:
##### y for the target varibale
##### X for the input features

In [8]:
x=df.drop('Exited',axis=1)
y=df['Exited']
print(x.shape,y.shape)

(10000, 10) (10000,)


In [10]:
num_col=x.select_dtypes(include='number').columns.to_list()
num_col

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [12]:
cat_col=x.select_dtypes(include='object').columns.to_list()
cat_col

['Geography', 'Gender']

In [14]:
def perc_target(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    df2=pd.DataFrame(np.array(kv).T,columns=['Exited','Count'])
    df2['Count']=df2['Count'].astype('int64')
    df2['%']=round(df2['Count']/a.shape*100,2)
    return df2.sort_values('Count',ascending=False)

In [16]:
perc_target(y)

,Exited,Count,%
0,0,7963,79.63
1,1,2037,20.37


## Create a Train Test Split
### We will continue with splitting our data into separate training and test sets.

##### 30% of observations will be set aside for the test set
##### the rest, 70%, will be used as the training set

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10)
print(len(x_train),len(x_test),len(y_train),len(y_test))

7000 3000 7000 3000


In [22]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 5663 to 1289
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      7000 non-null   int64  
 1   Geography        7000 non-null   object 
 2   Gender           7000 non-null   object 
 3   Age              7000 non-null   int64  
 4   Tenure           7000 non-null   int64  
 5   Balance          7000 non-null   float64
 6   NumOfProducts    7000 non-null   int64  
 7   HasCrCard        7000 non-null   int64  
 8   IsActiveMember   7000 non-null   int64  
 9   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 601.6+ KB


## Pre-processing Pipeline
### Scale numerical data and encode categorical data
##### Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder Create lists of indexes from the list of column namesNeed to be numeric not string to specify columns name in column transformer

In [25]:
num_feature=[]
for i in num_col:
    loc=x.columns.get_loc(i)
    num_feature.append(loc)

num_feature

[0, 3, 4, 5, 6, 7, 8, 9]

In [27]:
cat_feature=[]
for i in cat_col:
    loc=x.columns.get_loc(i)
    cat_feature.append(loc)

cat_feature

[1, 2]

In [29]:
preprocessor=make_column_transformer(
    (MinMaxScaler(),num_feature),
    (OneHotEncoder(sparse_output=False),cat_feature)
)
preprocessor

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [0, 3, 4, 5, 6, 7, 8, 9]),
                                ('onehotencoder',
                                 OneHotEncoder(sparse_output=False), [1, 2])])

In [33]:
from sklearn.ensemble import RandomForestClassifier
model=make_pipeline(
    preprocessor,
    SMOTE(random_state=10),
    RandomForestClassifier()
)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  [1, 2])])),
                ('smote', SMOTE(random_state=10)),
                ('randomforestclassifier', RandomForestClassifier())])

In [35]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'randomforestclassifier__n_estimators' : [50, 100, 150],
    'randomforestclassifier__max_features' : ['sqrt', 0.33],
    'randomforestclassifier__min_samples_leaf' : [1, 5 ,10, 15],
    'randomforestclassifier__criterion' : ['gini', 'entropy'],
    'randomforestclassifier__min_samples_split' : [2, 3, 4]
}
rf_grid=GridSearchCV(model,param_grid,verbose=3,cv=5,scoring='accuracy')

In [37]:
x_train=x_train.values
x_test=x_test.values

In [39]:
rf_grid.fit(x_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5] END randomforestclassifier__criterion=gini, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=50;, score=0.832 total time=   0.9s
[CV 2/5] END randomforestclassifier__criterion=gini, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=50;, score=0.834 total time=   0.5s
[CV 3/5] END randomforestclassifier__criterion=gini, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=50;, score=0.832 total time=   0.5s
[CV 4/5] END randomforestclassifier__criterion=gini, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_leaf=1, randomforestclassifier_

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [0, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8,
                                                                          9]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(sparse_output=False),
                                                                         [1,
                                                                          2])])),
                                       ('smote', SMOTE(random_state=10)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             param_grid={'randomforestclassifier__criterion': ['gini',
                                                               'entropy'],
                         'randomforestclassifier__max_features': ['sqrt', 0.33],
                         'randomforestclassifier__min_samples_leaf': [1, 5, 10,
                                                                      15],
                         'randomforestclassifier__min_samples_split': [2, 3, 4],
                         'randomforestclassifier__n_estimators': [50, 100,
                                                                  150]},
             scoring='accuracy', verbose=3)

In [41]:
print(rf_grid.best_params_)

{'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_features': 0.33, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 3, 'randomforestclassifier__n_estimators': 100}


In [43]:
print(rf_grid.best_score_)

0.8375714285714286


In [45]:
print(f'Training Data Score: {rf_grid.score(x_train,y_train)}')
print(f'Testining Data Score: {rf_grid.score(x_test,y_test)}')

Training Data Score: 0.999
Testining Data Score: 0.834


In [47]:
predictions=rf_grid.predict(x_test)
predictions

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [53]:
pd.DataFrame({'Actual':y_test,'Predictions':predictions}).reset_index(drop=True)

,Actual,Predictions
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
2995,0,0
2996,0,0
2997,0,0
2998,1,0


In [59]:
cm=confusion_matrix(y_test,predictions)
print(cm)

[[2110  244]
 [ 254  392]]


In [75]:
cm=np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.9  0.1 ]
 [0.39 0.61]]


In [77]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      2354
           1       0.62      0.61      0.61       646

    accuracy                           0.83      3000
   macro avg       0.75      0.75      0.75      3000
weighted avg       0.83      0.83      0.83      3000



In [79]:
import joblib
filename='../Models/RandomForest.sav'
joblib.dump(rf_grid,filename)

['../Models/RandomForest.sav']

In [81]:
rf_model=joblib.load(filename)
rf_model.score(x_test,y_test)

0.834